<a href="https://colab.research.google.com/github/velusrinathmr/agenticai/blob/main/Google_ADK_Multi_agent_TrainingAssess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of Google ADK and other dependencies

In [1]:
pip install google-adk -q

# ENV variables

## Imports

In [2]:
import os

## Env variable allocation

In [3]:
from google.colab import userdata
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
  print("No Google API Key found")
  GOOGLE_API_KEY = input("Enter Google API Key manually")

In [4]:
GOOGLE_GENAI_USE_VERTEXAI = "FALSE"

# Agent Development

## Imports

In [5]:
from google.adk.agents import Agent, LlmAgent

## Tools creation

### Flight search tool

In [6]:
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):

    flights = []

    if "london" in destination.lower():
        flights.append({"airline": "AirIndia", "price": 25000, "departure_time": "08:00", "arrival_time": "23:30"})
        flights.append({"airline": "Swiss Airlines", "price": 60000, "departure_time": "10:00", "arrival_time": "03:45"})
    elif "new york" in destination.lower():
        flights.append({"airline": "Air India", "price": 45000, "departure_time": "09:00", "arrival_time": "01:00"})
        flights.append({"airline": "Qatar Airways", "price": 50000, "departure_time": "11:00", "arrival_time": "07:30"})
    else:
        flights.append({"airline": "Emirates", "price": 30000, "departure_time": "07:00", "arrival_time": "12:00"})
    return flights

### Hotel search tool

In [7]:
def google_hotels(location: str, check_in_date: str, check_out_date: str, min_stars: int = 3):

    hotels = []

    if "london" in location.lower():
        hotels.append({"name": "Grand Stay Inn", "price_per_night": 150, "stars": 4, "amenities": ["Wifi", "Breakfast"]})
        hotels.append({"name": "Budget Lodge", "price_per_night": 80, "stars": 3, "amenities": ["Wifi"]})
    elif "new york" in location.lower():
        hotels.append({"name": "City View Suites", "price_per_night": 250, "stars": 4.5, "amenities": ["Wifi", "Gym", "Pool"]})
        hotels.append({"name": "Downtown Hostel", "price_per_night": 70, "stars": 2.5, "amenities": ["Shared Kitchen"]})
    else:
        hotels.append({"name": "Cozy Cottage", "price_per_night": 120, "stars": 4, "amenities": ["Wifi", "Parking"]})
    return hotels

### Activites search

In [8]:
def search_activities(location, interests):

    activities = []

    if "london" in location.lower():
        activities.append({"name": "British Museum Tour", "type": "Culture", "price": 0})
        activities.append({"name": "Thames River Cruise", "type": "Sightseeing", "price": 30})
        if interests and "food" in [i.lower() for i in interests]:
            activities.append({"name": "Borough Market Food Tour", "type": "Food", "price": 75})
    elif "new york" in location.lower():
        activities.append({"name": "Central Park Bike Rental", "type": "Outdoors", "price": 25})
        activities.append({"name": "Broadway Show Tickets", "type": "Entertainment", "price": 120})
    else:
        activities.append({"name": "Central Park", "type": "Outdoors", "price": 25})
    return activities

## Agent creation

In [62]:
llm_model = "gemini-1.5-flash"

In [63]:
flight_agent = LlmAgent(
    name="FlightAgent",
    model=llm_model,
    description="An agent specialized in finding flight information.",
    instruction=(
        "You are an expert in finding flights. Use the `search_flights` tool to find flight options. "
        "Always ask for origin, destination, and departure date. If return date is missing for a round trip, ask for it. "
        "Present flight options clearly, including airline, price, and times."
    ),
    tools=[search_flights],
)

In [64]:
hotel_agent = LlmAgent(
    name="HotelAgent",
    model=llm_model,
    description="An agent specialized in finding hotel accommodations.",
    instruction="You are an expert in finding hotels. Use the `google_hotels` tool to find hotel options.",
    tools=[google_hotels],
)

In [65]:
activity_agent = LlmAgent(
    name="ActivityAgent",
    model=llm_model,
    description="An agent specialized in suggesting activities and attractions.",
    instruction="Use `search_activities` to suggest local activities.",
    tools=[search_activities],
)

In [66]:
travel_coordinator = LlmAgent(
    name="TravelCoordinatorAgent",
    model=llm_model,
    description="Main coordinator for travel planning tasks.",
    instruction=(
        "Understand user travel goals and coordinate sub-agents (FlightAgent, HotelAgent, ActivityAgent). "
        "You can also ask for user's name and date details to ensure accurate information."
    ),
    sub_agents=[flight_agent, hotel_agent, activity_agent],
)

# Session and Runner setup

## Session setup

In [14]:
from google.adk.sessions import InMemorySessionService

In [15]:
APP_NAME="travel_planner_app"
USER_ID="user_1"
SESSION_ID="session_001"

In [16]:
session_service=InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

## Runner setup

In [17]:
from google.adk.runners import Runner

In [67]:
runner=Runner(
    agent=travel_coordinator,
    app_name=APP_NAME,
    session_service=session_service
)

# Testing Agent Interaction

In [19]:
import asyncio
from google.genai import types

In [68]:
prompt = "Plan a trip to Paris for 5 days"

final_response_text = "Agent did not produce a final response."



async def process_response(prompt):

    content = types.Content(role='user', parts=[types.Part(text=prompt)])

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                global final_response_text
                final_response_text = event.content.parts[0].text

await process_response(prompt)


In [69]:
from IPython.display import display, Markdown

display(Markdown(final_response_text))

I understand you'd like me to plan a 5-day trip to Paris.  However, to do so effectively, I need more information from you.  Please tell me:

1. **Your travel dates:**  When are you planning to go to Paris?  This significantly impacts flight and hotel prices and availability.
2. **Your budget:** What is your approximate budget for the entire trip, including flights, accommodation, activities, and food?
3. **Your travel style:** Are you looking for a luxurious trip, a budget-friendly adventure, a romantic getaway, a family vacation, or something else?
4. **Your interests:** What are you most interested in seeing and doing in Paris? (e.g., museums, historical sites, shopping, nightlife, food, etc.)
5. **Who are you traveling with?**  Are you traveling solo, with a partner, family, or friends?

Once I have this information, I can start creating a detailed itinerary for your 5-day trip to Paris.


In [70]:
prompt = """
Travel date - August 4,
Budget - 100000,
Travel style - budget friendly,
interests - nightlife,
travel companions - none,
"""

final_response_text = "Agent did not produce a final response."

await process_response(prompt)

display(Markdown(final_response_text))

Okay, a budget-friendly solo trip to Paris focusing on nightlife from August 4th with a $100,000 budget.  That's a very generous budget for a solo trip!  We can make this excellent.

To proceed, I need one more crucial piece of information:

* **Your departure city:** Knowing your departure city will allow me to find suitable and cost-effective flight options.

Once you provide your departure city, I will:

1. **Suggest flight options:** I'll look for flights that align with your budget, considering whether you'd prefer a standard or potentially a business class flight given the budget.
2. **Recommend accommodation:**  We can find a well-located, charming hotel or Airbnb in areas known for vibrant nightlife (Le Marais, Canal Saint-Martin, Latin Quarter).
3. **Create a nightlife itinerary:**  To tailor this best, tell me what *kind* of nightlife you enjoy (e.g., live music, clubs, bars, speakeasies, specific genres of music).  I'll then suggest specific venues.
4. **Suggest daytime activities:** Even with a nightlife focus, we'll include some daytime activities to avoid burnout.  These could be free or low-cost options like walking tours or visiting free museums on certain days.
5. **Plan your food:** I'll incorporate a mix of affordable and delicious meals, from casual bistros to occasional nicer dinners.

Let's get that departure city, and we can start crafting your amazing Parisian adventure!


In [72]:
prompt = """
i will be departing from Mumbai and I would like to return after a week from my departure.
prepare the travel plan with this information. you can skip remaining questions
"""

final_response_text = "Agent did not produce a final response."

await process_response(prompt)

display(Markdown(final_response_text))

Okay, here's a draft plan for your 5-day/6-night trip to Paris departing from Mumbai on August 4th and returning on August 10th, focusing on nightlife with a budget of $100,000.  Remember this is a *draft* and can be refined once we have confirmed flight and accommodation bookings.  Prices are estimates and may vary.


**Flights (Estimated $1500 - $5000):**

* I recommend checking flight comparison websites (e.g., Google Flights, Skyscanner, Kayak) for flights from Mumbai (BOM) to Paris (CDG or ORY).  Given your budget, you could explore options with premium economy or business class for increased comfort on the long journey.  The exact cost will depend on the specific dates and availability.

**Accommodation (Estimated $100 - $500 per night):**

* **Option 1 (Budget-friendly):**  A well-located Airbnb in Le Marais, Canal Saint-Martin, or the Latin Quarter.  These areas offer excellent nightlife and good access to public transportation.
* **Option 2 (Mid-range):**  A charming boutique hotel in the same areas.
* **Option 3 (Luxury -  only if significant budget remains after flights):** A higher-end hotel.


**Nightlife Itinerary (Example - customizable):**

* **Night 1:** Explore the bars and clubs around Canal Saint-Martin (known for its trendy and youthful vibe).
* **Night 2:** Check out live music venues in the Latin Quarter or Le Marais. (Research specific venues based on your preferred genres.)
* **Night 3:** Experience a more upscale cocktail bar in Le Marais or the Saint-Germain-des-Prés district.
* **Night 4:** Discover hidden speakeasies (research beforehand for recommendations).
* **Night 5:** Relax and enjoy a wine bar with a view of the city.
* **Night 6 (Departure Day):** Enjoy a final Parisian breakfast before heading to the airport.

**Daytime Activities (Example):**

* **Day 1:** Arrive in Paris, check into your accommodation, and take a leisurely walk to familiarize yourself with your surroundings.
* **Day 2:** Explore the Louvre Museum (or another museum of your interest), followed by a picnic in the Tuileries Garden.
* **Day 3:** Visit Montmartre, explore the Sacré-Cœur Basilica, and enjoy the artistic atmosphere.
* **Day 4:**  Take a boat tour on the Seine River, followed by exploring a specific district or neighborhood based on your interest.
* **Day 5:**  Visit a market, do some souvenir shopping, or relax in a park before heading to the airport.


**Food (Estimated $50 - $150 per day):**

* Mix of casual bistros, street food, and occasional nicer restaurants.

**Transportation:**

* Utilize the efficient and affordable Paris Metro system.

**Total Estimated Cost (excluding flights):**  $500 (accommodation) + $750 (food) + $300 (activities and transport) = $1550  This is a *rough estimate*, and the actual cost will vary based on your choices.

**Next Steps:**

1. **Book Flights:** Use online comparison sites to find flights that fit your budget and schedule.
2. **Book Accommodation:**  Reserve your Airbnb or hotel based on your preferred style and location.
3. **Research Nightlife Venues:**  Explore options based on your preferences (music genres, atmosphere).
4. **Plan Daytime Activities:**  Research museums, parks, and attractions that interest you.

This is a flexible plan. We can refine it further once you've booked flights and accommodation.  Remember, your generous budget allows for flexibility and potentially higher-end experiences if desired. Let me know if you'd like any adjustments or further details.
